<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.


Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [ ]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 35.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp310-cp310-linux_x86_64.whl size=1192338 sha256=14d5ed08c3bce49e4800b1ec779089c40f1703bd1f3c5d9646c81040c37a6d30
  Stored in directory: /root/.cache/pip/wheels/8b/6d/9c/e1793e5a2d85b89b8c362a4f0e3def54b786bcd14923b9ff00
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.20
    Uninstalling SQLAlchemy-2.0.20:
      Successfully uninstalled SQLAlchemy-2.0.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [80]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [81]:
import csv, sqlite3

con = sqlite3.connect("tatabase.db")

cur = con.cursor()

In [82]:
#!pip install -q pandas==1.1.5

In [83]:
%sql sqlite:///tatabase.db

In [84]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [85]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///tatabase.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [86]:
display = %sql select distinct "Launch_Site" from SPACEXTABLE
display

 * sqlite:///tatabase.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA'


In [ ]:
query= %sql select * from SPACEXTABLE where Launch_Site like "%CCA%";
query


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [88]:
query = %sql select sum(PAYLOAD_MASS__KG_) from SPACEXTABLE where Customer like "NASA (CRS)%"
query

 * sqlite:///tatabase.db
Done.


sum(PAYLOAD_MASS__KG_)
48213


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [89]:
query = %sql select avg(PAYLOAD_MASS__KG_) from SPACEXTABLE where Booster_Version like "%F9 v1.1%";
query

 * sqlite:///tatabase.db
Done.


avg(PAYLOAD_MASS__KG_)
2534.6666666666665


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_


In [117]:
query= %sql select min(date(Date)) as "Date" from SPACEXTABLE;
query

 * sqlite:///tatabase.db
Done.


Date
2010-04-06


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [ ]:
query= %sql select Booster_Version from SPACEXTABLE where Mission_Outcome = "Success" \
and (select PAYLOAD_MASS__KG_ from SPACEXTABLE where PAYLOAD_MASS__KG_ between 4000 and 6000)
# %sql select *  from SPACEXTABLE limit 20;
query

### Task 7




##### List the total number of successful and failure mission outcomes


In [92]:
query = %sql select case when Mission_Outcome like "%Success%" then "success" else "failure" end as new_outcome, count(*) as freq from \
SPACEXTABLE where Mission_Outcome like "%Success%" or Mission_Outcome like "%Fail%" group by new_outcome
query

 * sqlite:///tatabase.db
Done.


new_outcome,freq
failure,1
success,100


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [ ]:
query = %sql select Booster_Version from SPACEXTABLE where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTABLE);
query

### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.**


In [114]:
query=%sql select case WHEN substr(Date,6,2) = "01" THEN "jan" WHEN substr(Date,6,2) = "02" THEN "feb" \
WHEN substr(Date,6,2) = "03" THEN "mar" WHEN substr(Date,6,2) = "04" THEN "apr" \
WHEN substr(Date,6,2) = "05" THEN "may" WHEN substr(Date,6,2) = "06" THEN "jun" \
WHEN substr(Date,6,2) = "07" THEN "jul" WHEN substr(Date,6,2) = "08" THEN "aug" \
WHEN substr(Date,6,2) = "09" THEN "sep" WHEN substr(Date,6,2) = "10" THEN "oct" \
WHEN substr(Date,6,2) = "11" THEN "nov" else "dec" end as "Month", Booster_Version , Landing_Outcome , Date from SPACEXTABLE \
where Landing_Outcome like "Failure (drone ship)" and substr(Date,1,4) = "2015";
query

 * sqlite:///tatabase.db
Done.


Month,Booster_Version,Landing_Outcome,Date
oct,F9 v1.1 B1012,Failure (drone ship),2015-10-01
apr,F9 v1.1 B1015,Failure (drone ship),2015-04-14


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [142]:
%sql create table new_table as select count(*) as freq, Landing_Outcome as Outcome from SPACEXTABLE where date(Date) between '2010-06-04' and '2017-03-20' group by Landing_Outcome order by freq Desc;
%sql create table ranking("rank" varchar(10), "Outcome" varchar(255)); insert into ranking(rank,Outcome) values ("1st",	"No attempt"), \
("2nd",	"Success (ground pad)"),("2nd",	"Success (drone ship)"),("3rd",	"Failure (drone ship)"),("4th",	"Controlled (ocean)"),("5th",	"Uncontrolled (ocean)"), \
("6th",	"Precluded (drone ship)"),("6th",	"Failure (parachute)");



 * sqlite:///tatabase.db
Done.
 * sqlite:///tatabase.db
Done.
8 rows affected.


[]

In [147]:
query=%sql select rk.rank, nt.outcome, nt.freq from new_table as nt inner join ranking as rk on nt.Outcome = rk.Outcome
query

 * sqlite:///tatabase.db
Done.


rank,Outcome,freq
1st,No attempt,10
2nd,Success (ground pad),5
2nd,Success (drone ship),5
3rd,Failure (drone ship),5
4th,Controlled (ocean),3
5th,Uncontrolled (ocean),2
6th,Precluded (drone ship),1
6th,Failure (parachute),1


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
